#### Before You Start


cd .devcontainer </br>
run docker-compose up

In [ ]:
! pip install psycopg2
! pip install sqlalchemy


In [ ]:
import psycopg2  #import of the psycopg2 python library
import pandas as pd #import of the pandas python library
import pandas.io.sql as psql

##No transaction is started when commands are executed and no commit() or rollback() is required. 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [ ]:
try:
    # Connect to the postgreSQL server with username, and password credentials
    con = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "localhost",
                                  port = "5432")
    
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print("Connected Successfully to PostgreSQL server!!")
    
    # Obtain a DB Cursor to perform database operations
    cursor = con.cursor();
except (Exception, psycopg2.Error) as error :
     print ("Error while connecting to PostgreSQL", error)


### Test that reading CSV works

In [ ]:
import pandas as pd
df = pd.read_csv('circuits.csv')
print(df.head())  # Displays the first 5 rows


### Create sample table

In [ ]:
#Create "test" Table

try:
    #table_name variable
    seasonTable="season"
    create_all_tables_query = '''
    CREATE TABLE IF NOT EXISTS circuit (
        circuit_id INT PRIMARY KEY,
        name TEXT,
        location TEXT,
        country TEXT,
        lat FLOAT,
        long FLOAT,
        alt INT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS constructor (
        constructor_id INT PRIMARY KEY,
        name TEXT,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS driver (
        driver_id INT PRIMARY KEY,
        code TEXT,
        first_name TEXT,
        last_name TEXT,
        dob DATE,
        nationality TEXT,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS race (
        race_id INT PRIMARY KEY,
        year INT,
        round INT,
        circuit_id INT REFERENCES circuit,
        name TEXT,
        date DATE,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS season (
        year INT PRIMARY KEY,
        url TEXT
    );

    CREATE TABLE IF NOT EXISTS constructor_result (
        constructor_results_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        constructor_id INT REFERENCES constructor,
        points INT 
    );

    CREATE TABLE IF NOT EXISTS constructor_standing (
        constructor_standings_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        constructor_id INT REFERENCES constructor,
        points INT,
        position INT,
        wins INT
    );

    CREATE TABLE IF NOT EXISTS driver_standing (
        driver_standings_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        driver_id INT REFERENCES driver,
        points INT,
        position INT,
        wins INT
    );

    CREATE TABLE IF NOT EXISTS lap_time (
        race_id INT PRIMARY KEY,
        driver_id INT REFERENCES driver,
        lap INT,
        position INT,
        time_in_miliseconds INT
    );

    CREATE TABLE IF NOT EXISTS pit_stop (
        race_id INT,
        driver_id INT,
        PRIMARY KEY (race_id, driver_id),
        stop INT,
        lap INT,
        time_in_miliseconds INT
    );

    CREATE TABLE IF NOT EXISTS result_id (
        result_id INT PRIMARY KEY,
        race_id INT REFERENCES race,
        driver_id INT REFERENCES driver,
        constructor_id INT REFERENCES constructor,
        position INT,
        points INT,
        laps INT,
        time_in_miliseconds INT,
        fastestLap INT,
        rank INT,
        fastest_lap_time INT,
        fastest_lap_speed FLOAT
    );
    '''


    #Execute this command (SQL Query)
    cursor.execute(create_all_tables_query)
    
    # Make the changes to the database persistent
    con.commit()
    print("All tables created successfully in PostgreSQL ")
except (Exception, psycopg2.Error) as error:
    # if it exits with an exception the transaction is rolled back.
    con.rollback()
    print("Error While Creating the DB: ",error)

Check table creation

In [ ]:
# [information_schema.tables] keep listing of every table being managed by Postgres for a particular database.
# specifying the tabel_schema to 'public' to only list tables that you create.
cursor.execute("""SELECT table_name 
                  FROM information_schema.tables 
                  WHERE table_schema = 'public'  
               """)

for table in cursor.fetchall():
    print(table)

### Test entering values manually

In [ ]:
# sql_insert_customers = "INSERT INTO season (year,url) VALUES(%s,%s)"

# #list of customers (With Their attributes values)
# #None here means value will be missing in the table
# customer_List=[
#             (200000, "mohamed.ragb@ut.ee"),
#             (2, "j.smith@hotmail.com"),
#             (3, None),
#             (4, "jeanlime@gmail.com"),
#             (5,None)]

# try:
#     # execute the INSERT statement
#     cursor.executemany(sql_insert_customers,customer_List)
#     # commit the changes to the database
#     con.commit()
#     #the number of inserted rows/tuples
#     count = cursor.rowcount
#     print (count, "Record inserted successfully into customers table")

# except (Exception, psycopg2.Error) as error :
#     con.rollback()
#     print ("Error while Inserting the data to the table, Details: ",error)

### View manual inserts

In [ ]:
# sql_select_query = """ SELECT * FROM racestest """

# try:
    
#     cursor.execute(sql_select_query, (1,))
#     person_records = cursor.fetchall() 
#     print("Print each row and it's columns values:\n")
#     for row in person_records:
#         print("year = ", row[0], )
#         print("url = ", row[1], )
# except(Exception, psycopg2.Error) as error :
#     con.rollback()
#     print("Error:", error)

### Read seasons.CSV and insert into already created tables with constraints

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Load the CSV file into a pandas DataFrame
csv_file_path = "seasons.csv"  # Replace with your CSV file path
table_name = "season"  # Replace with your PostgreSQL table name
df = pd.read_csv(csv_file_path)

# Connect to the PostgreSQL database
# Replace the placeholders with your database credentials
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Write the DataFrame to the PostgreSQL table
try:
    df.to_sql(table_name, con=engine, if_exists="append", index=False)
    print(f"Data from {csv_file_path} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Load the CSV file into a pandas DataFrame
csv_file_path = "circuits.csv"  # Replace with your CSV file path
table_name = "circuit"  # Replace with your PostgreSQL table name
df = pd.read_csv(csv_file_path)

# Rename DataFrame columns to match the PostgreSQL table schema
df.rename(columns={
    "circuitId": "circuit_id",
    "lng": "long",
    # Add any other necessary mappings here
}, inplace=True)

# Write the DataFrame to the PostgreSQL table
try:
    df.to_sql(table_name, con=engine, if_exists="append", index=False)
    print(f"Data from {csv_file_path} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred: {e}")


### Read races.CSV and insert into race

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Load the CSV file into a pandas DataFrame
csv_file_path = "races.csv"  # Replace with your CSV file path
table_name = "race"  # Replace with your PostgreSQL table name
df = pd.read_csv(csv_file_path)

# Rename DataFrame columns to match the PostgreSQL table schema
df.rename(columns={
    "raceId": "race_id",
    "circuitId": "circuit_id",
    # Add any other necessary mappings here
}, inplace=True)

columns_to_keep = ["race_id", "year", "round", "circuit_id", "name", "date", "url"]
df = df[columns_to_keep]

# Connect to the PostgreSQL database
# Replace the placeholders with your database credentials
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Write the DataFrame to the PostgreSQL table
try:
    df.to_sql(table_name, con=engine, if_exists="append", index=False)
    print(f"Data from {csv_file_path} successfully imported into {table_name}!")
except Exception as e:
    print(f"An error occurred: {e}")


### View query

In [ ]:
# sql_select_query = """ SELECT * FROM season WHERE year = 2001"""
# sql_select_query = """ SELECT r.race_id, r.year FROM race as r JOIN season as s ON s.year = r.year WHERE s.year = 2001"""

# try:
#     cursor.execute(sql_select_query, (1,))
#     person_records = cursor.fetchall() 
#     print("Print each row and it's columns values:\n")
#     # print(person_records)
#     df = pd.DataFrame(person_records, columns=['race_id', 'year'])
#     print(df)
# except(Exception, psycopg2.Error) as error :
#     con.rollback()
#     print("Error:", error)


Countries_Customers_Cnt_gt1= psql.read_sql("""SELECT r.race_id, r.year 
                                           FROM race as r 
                                           JOIN season as s 
                                           ON s.year = r.year 
                                           WHERE s.year = 2001
                                          """, engine)
display(Countries_Customers_Cnt_gt1.style)